In [1]:
pip install -q "tensorflow-text==2.8.*"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import Word2Vec

In [3]:
#Open data
data_file = open('Data.txt').read()
data_file

'The important thing is not to stop questioning. Curiosity has its own reason for existence. One cannot help but be in awe when he contemplates the mysteries of eternity, of life, of the marvelous structure of reality. It is enough if one tries merely to comprehend a little of this mystery each day.'

In [4]:
#Create tokenizer object
tokenizer = Tokenizer()

#Convert data to lowercase
data = data_file.lower().split('.')

In [5]:
#Create dictionary of words with the frequency they occur
#Every word gets unique value > 0
#0 is reserved for padding
tokenizer.fit_on_texts(data)

#Transforms sentences into set of integers from the dictionary
sequences = tokenizer.texts_to_sequences(data)

#Counts total words
total_words = len(tokenizer.word_index) + 1

#Prints it all out
print(tokenizer.word_index, '\n')
print(sequences, '\n')
print(total_words, '\n')

{'of': 1, 'the': 2, 'is': 3, 'to': 4, 'one': 5, 'important': 6, 'thing': 7, 'not': 8, 'stop': 9, 'questioning': 10, 'curiosity': 11, 'has': 12, 'its': 13, 'own': 14, 'reason': 15, 'for': 16, 'existence': 17, 'cannot': 18, 'help': 19, 'but': 20, 'be': 21, 'in': 22, 'awe': 23, 'when': 24, 'he': 25, 'contemplates': 26, 'mysteries': 27, 'eternity': 28, 'life': 29, 'marvelous': 30, 'structure': 31, 'reality': 32, 'it': 33, 'enough': 34, 'if': 35, 'tries': 36, 'merely': 37, 'comprehend': 38, 'a': 39, 'little': 40, 'this': 41, 'mystery': 42, 'each': 43, 'day': 44} 

[[2, 6, 7, 3, 8, 4, 9, 10], [11, 12, 13, 14, 15, 16, 17], [5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2, 27, 1, 28, 1, 29, 1, 2, 30, 31, 1, 32], [33, 3, 34, 35, 5, 36, 37, 4, 38, 39, 40, 1, 41, 42, 43, 44], []] 

45 



In [6]:
#Pad sequences
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen = 20, padding ='post', value = 0)

In [39]:
#sequences = np.asarray(sequences)
sequences = np.array(sequences, dtype=np.int64)
sequences = np.ndarray.flatten(sequences)
sequences.shape

(100,)

In [57]:
#Convert to embedding vectors
embedding_model = keras.Sequential()

embedding_model.add(keras.layers.Embedding(100, 512, mask_zero = True, input_length = 100))

embedding_model.add(keras.layers.Embedding(100, 512, mask_zero = True, input_length = 512))

embedding_model.add(keras.layers.Embedding(100, 512, mask_zero = True, input_length = 512))

embedding_model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Nadam(), metrics=[keras.metrics.SparseCategoricalAccuracy()])
                    
keras.utils.plot_model(embedding_model,to_file='model.png', show_shapes=True,expand_nested=True)

history = embedding_model.fit(
  sequences, sequences,
  batch_size=1,
  epochs=100)
word_vectors = embedding_model.predict(sequences)
word_vectors

Epoch 1/100
 36/100 [=========>....................] - ETA: 26s - loss: 281.3391 - sparse_categorical_accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
#Calculate positional encoding
